In [ ]:
try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import requests
    import json
except ImportError:
    %pip install pandas numpy matplotlib requests json
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import requests
    import json


In [ ]:
import subprocess
cmd_status = ["docker", "compose", "-f", "docker-compose.yaml", "ps", "-q"]
if not subprocess.run(cmd_status, capture_output=True, text=True).stdout.strip():
    subprocess.run(["docker", "compose", "-f", "docker-compose.yaml", "up", "-d"], check=True)
    print("Docker Compose started.")
else:
    print("Docker Compose already running.")


# ==========================================
# 1. THE "PLANT" (Simulation Environment)
# ==========================================

In [ ]:
class SmartHomeEnv:
    def __init__(self, battery_capacity=10.0, max_power=3, seed=42, hours=72):
        self.battery_capacity = battery_capacity # kWh
        self.max_power = max_power # kW
        self.soc = 5 # Initial State of Charge (kWh)
        self.initial_soc = self.soc
        self.time_step = 0

        # Set a fixed seed for reproducibility
        np.random.seed(seed)

        # Internal Data Generation (The "Real World")
        # We now use the minute-level data as the primary simulation data
        self.data_hourly, self.data = self._generate_scenario_data(hours=hours)

    def _generate_scenario_data(self, hours):
        minutes = hours * 60
        t_min = np.arange(minutes)
        t_hours = t_min / 60.0

        # Solar: Peak at noon + random clouds
        solar = np.maximum(0, 5 * np.sin(2 * np.pi * (t_hours - 6) / 24))
        solar = np.maximum(0, solar - 0.3 * np.random.weibull(0.5, size=minutes))

        # Load: Morning/Evening peaks
        load = 2 + np.cos(4 * np.pi * (t_hours - 18) / 24) + \
                0.8 * np.cos(2 * np.pi * (t_hours - 14) / 24)
        load = np.maximum(0.5, load)
        # Price: High in evening
        price = 0.20 + 0.3 * np.cos(2 * np.pi * (t_hours - 18) / 24)
        
        # Sell Price: Fixed feed-in tariff (e.g. 0.10 EUR/kWh)
        sell_price = np.minimum(price,0.10 * np.ones(minutes)) - 0.1
        # sell_price = price - 0.1

        df_min = pd.DataFrame({'solar': solar, 'load': load, 'price': price, 'sell_price': sell_price})
        
        # Aggregate to hourly
        # We group by integer division of index by 60 to get hourly blocks
        df_hourly = df_min.groupby(df_min.index // 60).mean()

        return df_hourly, df_min

    def reset(self):
        self.soc = self.initial_soc
        self.time_step = 0
        return self.data.iloc[0]

    def step(self, action_kw):
        """
        Executes one time step.
        Args:
            action_kw (float): Desired battery power (+ Charge, - Discharge)
        Returns:
            observation (Series): The NEXT state (load, solar, price)
            reward (float): The cost incurred this step
            done (bool): Is simulation over?
            info (dict): Debug info
        """
        current_data = self.data.iloc[self.time_step]

        # --- 1. Apply Physics Constraints (The "Real" Battery) ---
        # A. Power Limits
        power = np.clip(action_kw, -self.max_power, self.max_power)

        # B. Capacity Limits
        # Energy change is Power * Time. Time is 1 minute (1/60 hour).
        energy_change = power / 60.0
        
        if energy_change > 0: # Charging
            max_charge = self.battery_capacity - self.soc
            # Limit energy, then convert back to power limit if needed, 
            # but simpler to just limit the energy added.
            if energy_change > max_charge:
                energy_change = max_charge
                power = energy_change * 60.0
        else: # Discharging
            max_discharge = self.soc
            if -energy_change > max_discharge:
                energy_change = -max_discharge
                power = energy_change * 60.0

        # --- 2. Update State ---
        self.soc += energy_change

        # --- 3. Calculate Cost ---
        # Grid Balance: Load + Charge = Solar + Discharge + Grid
        # Grid = (Load - Solar) + Power
        net_load = current_data['load'] - current_data['solar']
        grid_kw = net_load + power

        # Cost is for the energy consumed in this minute
        grid_kwh = grid_kw / 60.0
        
        if grid_kw > 0:
            cost = grid_kwh * current_data['price']
        else:
            cost = grid_kwh * current_data['sell_price']

        # --- 4. Prepare Next Step ---
        self.time_step += 1
        done = self.time_step >= len(self.data)

        next_obs = None
        if not done:
            next_obs = self.data.iloc[self.time_step]

        info = {
            'soc': self.soc,
            'grid_kw': grid_kw,
            'battery_action_actual': power,
            'load': current_data['load'],
            'solar': current_data['solar'],
            'price': current_data['price'],
            'sell_price': current_data['sell_price']
        }

        return next_obs, cost, done, info

    def get_forecast(self, horizon=24):
        """Returns the data for the next N steps (minutes)"""
        start = self.time_step
        end = min(start + horizon, len(self.data))
        return self.data.iloc[start:end]

    def get_hourly_forecast(self, horizon=24):
        """Returns the hourly data for the next N hours"""
        current_hour = self.time_step // 60
        start = current_hour
        end = min(start + horizon, len(self.data_hourly))
        return self.data_hourly.iloc[start:end]

# ==========================================
# 2. THE OPTIMIZERS (Hourly)
# ==========================================

In [ ]:
class Optimizer:
    """Base optimizer: stores battery capacity and max power values and defines interface."""
    def __init__(self, env):
        self.bat_cap = env.battery_capacity
        self.max_p = env.max_power
        self.env = env

    def get_setpoint(self, observation, current_soc):
        """
        Calculate the setpoint for the next hour.
        Args:
            observation: Hourly aggregated data for the current hour.
            current_soc: Current State of Charge.
        Returns:
            float: Battery power setpoint (kW) for the next hour.
        """
        raise NotImplementedError

In [ ]:
class ResidualChargeOptimizer(Optimizer):
    """Store excess solar; discharge to meet deficits (Hourly Average)."""

    def get_setpoint(self, observation, current_soc):
        net_load = observation['load'] - observation['solar']
        if net_load < 0:
            return min(self.max_p, -net_load)
        else:
            return -min(self.max_p, net_load)

In [ ]:
class UrbsMPCOptimizer(Optimizer):
    """MPC optimizer using Urbs for optimization with perfect foresight."""
    def __init__(self, env, url="http://localhost:5000/simulate"):
        super().__init__(env)
        self.url = url
        self.plan = None
        self.horizon = 10000
        self.plan_initial_soc = None

    def get_setpoint(self, observation, current_soc):
        # Plan once at the beginning (Perfect Foresight)
        if self.plan is None:
            self.plan = self._run_optimization(horizon=self.horizon, initial_soc=current_soc)
            self.plan_initial_soc = current_soc
        
        # Get action for current timestep (Hour)
        t_hour = self.env.time_step // 60
        
        if t_hour < len(self.plan):
            # Closed-Loop execution (SoC Tracking) to prevent drift:
            # Calculate where we SHOULD be at the end of this step according to the plan
            # Target SoC = Initial + Sum of all planned actions up to and including this step
            target_soc_end = self.plan_initial_soc + np.sum(self.plan[:t_hour+1])
            
            # Action required to get from current_soc to target_soc_end
            # Since we are planning for 1 hour, Energy (kWh) = Power (kW) * 1h
            action_required = target_soc_end - current_soc
            
            return action_required
        else:
            return 0.0

    def _run_optimization(self, horizon=10000, initial_soc=0.0):
        # 1. Get Full Forecast (Hourly)
        full_data = self.env.get_hourly_forecast(horizon=horizon)
        timesteps = len(full_data)
        
        if timesteps < 2:
            return np.zeros(timesteps)

        # 2. Construct JSON Payload
        # Normalize Solar: Urbs SupIm is usually a profile. 
        # We'll set installed capacity to max(solar) and profile to solar/max.
        solar_profile = full_data['solar'].values
        max_solar = solar_profile.max()
        if max_solar == 0: max_solar = 1.0
        norm_solar = (solar_profile / max_solar).tolist()
        
        load_profile = full_data['load'].tolist()
        price_profile = full_data['price'].tolist()
        sell_price_profile = full_data['sell_price'].tolist()
        
        payload ={
            "site": {
                "Main": {
                    "area": 100,
                    "process": {
                        "Purchase": {
                            "wacc": 0,
                            "cap-lo": 0,
                            "cap-up": 1000,
                            "fix-cost": 0,
                            "inst-cap": 1000,
                            "inv-cost": 0,
                            "max-grad": "inf",
                            "var-cost": 0,
                            "commodity": {
                                "Elec": {
                                    "ratio": 1,
                                    "Direction": "Out",
                                    "ratio-min": 1
                                },
                                "Elec buy": {
                                    "ratio": 1,
                                    "Direction": "In",
                                    "ratio-min": 1
                                }
                            },
                            "description": "Buy electricity from the utility grid",
                            "depreciation": 50,
                            "min-fraction": 0
                        },
                        "Feed-in": {
                            "wacc": 0,
                            "cap-lo": 0,
                            "cap-up": 1000,
                            "fix-cost": 0,
                            "inst-cap": 1000,
                            "inv-cost": 0,
                            "max-grad": "inf",
                            "var-cost": 0,
                            "commodity": {
                                "Elec": {
                                    "ratio": 1,
                                    "Direction": "In",
                                    "ratio-min": 1
                                },
                                "Elec sell": {
                                    "ratio": 1,
                                    "Direction": "Out",
                                    "ratio-min": 1
                                }
                            },
                            "description": "Sell electricity to the utility grid",
                            "depreciation": 50,
                            "min-fraction": 0
                        },
                        "Photovoltaics": {
                            "wacc": 0.07,
                            "cap-lo": max_solar,
                            "cap-up": max_solar,
                            "fix-cost": 0,
                            "inst-cap": max_solar,
                            "inv-cost": 0,
                            "max-grad": "inf",
                            "var-cost": 0,
                            "commodity": {
                                "Elec": {
                                    "ratio": 1,
                                    "Direction": "Out",
                                    "ratio-min": 1
                                },
                                "Solar": {
                                    "ratio": 1,
                                    "Direction": "In",
                                    "ratio-min": 1
                                }
                            },
                            "description": "Generates electricity from sun",
                            "area-per-cap": 5,
                            "depreciation": 25,
                            "min-fraction": 0
                        }
                    },
                    "commodity": {
                        "Elec": {
                            "Type": "Demand",
                            "unitC": "kWh",
                            "unitR": "kW",
                            "demand": load_profile,
                            "storage": {
                                "Lead-Acid Battery": {
                                    "init": initial_soc / self.bat_cap if self.bat_cap > 0 else 0,
                                    "wacc": 0.007,
                                    "eff-in": 1,
                                    "eff-out": 1,
                                    "cap-lo-c": self.bat_cap,
                                    "cap-lo-p": self.max_p,
                                    "cap-up-c": self.bat_cap,
                                    "cap-up-p": self.max_p,
                                    "discharge": 0,
                                    "fix-cost-c": 0,
                                    "fix-cost-p": 0,
                                    "inst-cap-c": self.bat_cap,
                                    "inst-cap-p": self.max_p,
                                    "inv-cost-c": 0,
                                    "inv-cost-p": 0,
                                    "var-cost-c": 0,
                                    "var-cost-p": 0,
                                    "description": "Lead-Acid battery",
                                    "depreciation": 5
                                }
                            }
                        },
                        "Solar": {
                            "Type": "SupIm",
                            "supim": norm_solar,
                            "unitC": "kWh",
                            "unitR": "kW"
                        },
                        "Elec buy": {
                            "max": "inf",
                            "Type": "Buy",
                            "price": 0.1,
                            "unitC": "kWh",
                            "unitR": "kW",
                            "maxperhour": "inf"
                        },
                        "Elec sell": {
                            "max": "inf",
                            "Type": "Sell",
                            "price": 0.0,
                            "unitC": "kWh",
                            "unitR": "kW",
                            "maxperhour": "inf"
                        }
                    }
                }
            },
            "global": {
                "CO2 limit": 150000000,
                "Cost limit": 35000000000
            },
            "c_timesteps": timesteps,
            "buysellprice": {
                "Elec buy": price_profile,
                "Elec sell": sell_price_profile
            }
        }
        # 3. Send Request
        # try:

        # Save payload to JSON file
        with open('adg_payload.urbs', 'w') as f:
            json.dump(payload, f, indent=2)
        response = requests.post(self.url, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        
        # 4. Parse Result
        # result['results']['Main']['Elec']['storage']['Stored'] (Charge)
        # result['results']['Main']['Elec']['storage']['Retrieved'] (Discharge)
        storage_res = result['data']['results']['Main']['Elec']['storage']
        charge = np.array(storage_res['Stored'])
        discharge = np.array(storage_res['Retrieved'])
        

        # Net action: Charge - Discharge
        # Note: SmartHomeEnv expects positive for Charge, negative for Discharge.
        actions = charge - discharge
        return actions
            
        # except Exception as e:
        #     print(f"Optimization failed: {e}")
        #     # Fallback: Do nothing
        #     return np.zeros(timesteps)

In [ ]:
class LimitedURBSOptimizer(UrbsMPCOptimizer):
    """MPC optimizer using Urbs for optimization with perfect foresight and limited horizon."""
    def __init__(self, env, url="http://localhost:5000/simulate"):
        super().__init__(env, url)
        self.horizon = 1000

    def get_setpoint(self, observation, current_soc):
        actions = self._run_optimization(horizon=self.horizon, initial_soc=current_soc)
        
        # We only execute the first action of the plan
        if len(actions) > 0:
            return actions[0]
        else:
            return 0.0

# ==========================================
# 3. REAL-TIME CONTROLLER (Minute-Level)
# ==========================================

In [ ]:
class RealTimeController:
    """
    Runs every minute to execute the hourly setpoint.
    """
    def __init__(self, env):
        self.env = env
    
    def get_action(self, setpoint_kw, observation):
        """
        Determine the minute-level action.
        Args:
            setpoint_kw (float): The target battery power for the current hour (from Optimizer).
            observation (Series): Current minute-level observation (load, solar, etc.).
        Returns:
            float: Battery power action (kW) for this minute.
        """
        # For now, we simply try to follow the setpoint.
        # In a more complex scenario, we might adjust this based on 
        # instantaneous load/solar to maintain a grid exchange target, 
        # but here the setpoint IS the battery power target.
        return setpoint_kw


# ==========================================
# 4. Experiment
# ==========================================

In [ ]:

def run_experiment(OptimizerClass):
    # Setup
    env = SmartHomeEnv(hours=24)
    optimizer = OptimizerClass(env)
    rt_controller = RealTimeController(env)

    # History Storage
    results = []

    # Start
    obs = env.reset()
    done = False
    
    hourly_setpoint = 0.0

    print(f"Starting Simulation with {OptimizerClass.__name__}...")

    while not done:
        # 1. Hourly Optimization (Run once at the start of each hour)
        if env.time_step % 60 == 0:
            current_hour = env.time_step // 60
            # Get hourly observation for the UPCOMING hour
            if current_hour < len(env.data_hourly):
                obs_hourly = env.data_hourly.iloc[current_hour]
                # Optimizer decides setpoint for this hour
                hourly_setpoint = optimizer.get_setpoint(obs_hourly, env.soc)
            else:
                hourly_setpoint = 0.0

        # 2. Real-Time Control (Every Minute)
        action_requested = rt_controller.get_action(hourly_setpoint, obs)

        # 3. Environment reacts
        next_obs, cost, done, info = env.step(action_requested)

        # 4. Store Data
        info['action_requested'] = action_requested
        info['setpoint'] = hourly_setpoint
        info['cost'] = cost
        # info['soc'] is already in info from env.step
        results.append(info)

        # 5. Advance
        obs = next_obs

    # Process Results
    df_res = pd.DataFrame(results)
    return df_res

# ==========================================
# 5. VISUALIZATION
# ==========================================

In [ ]:
def plot_controller_performance(df, controller_name, total_cost):
    print(f"Total Cost for {controller_name}: €{total_cost:.2f}")

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    # Ax1: Physics
    ax1.set_title(f"{controller_name}: Power Flows")
    ax1.plot(df['load'], 'k--', label='Load', alpha=0.5)
    ax1.plot(df['solar'], 'orange', label='Solar', alpha=0.5)
    ax1.bar(df.index, df['battery_action_actual'], color='green', alpha=0.3, label='Battery Flow')
    ax1.legend()
    ax1.set_ylabel("kW")

    # Ax2: Battery State
    ax2.set_title(f"{controller_name}: Battery State of Charge")
    ax2.plot(df['soc'], 'g-', linewidth=2)
    ax2.set_ylabel("kWh")
    ax2.set_xlabel("Hour")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# ==========================================
# 6. COMPARISON
# ==========================================

In [ ]:
controllers = {
    "ResidualCharge": ResidualChargeOptimizer,
    "OneShotURBS": UrbsMPCOptimizer,
    "LimitedURBS": LimitedURBSOptimizer,
}

all_results = {}
for name, controller in controllers.items():
    all_results[name] = run_experiment(controller)


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
# Use the first result to get the index (which is now minute-level)
df = list(all_results.values())[0]

# Plotting minute-level data might be dense, but let's see.
# We might want to resample for visualization if it's too crowded.
ax1.plot(df['solar'], label='Solar', alpha=0.5)
ax1.plot(df['load'], 'k--', label='Load', alpha=0.5)
ax1.plot(df['price'], label='Price', alpha=0.5)
# ax1.plot(df['sell_price'], label='Sell Price')

for name, df in all_results.items():
    total_cost = df['cost'].sum()
    
    # Ax1: Physics
    ax1.set_title("Power Flows (Minute Resolution)")
    # Plotting battery action as a line might be clearer than bars for dense data
    ax1.plot(df.index, df['battery_action_actual'], alpha=0.6, label=f'{name} (Cost: {total_cost:.2f}€)')
    
    # Ax2: Battery State
    ax2.set_title("Battery State of Charge")
    ax2.plot(df['soc'], linewidth=2, label=f"{name}")

ax1.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax1.set_ylabel("kW")

ax2.set_ylabel("kWh")
ax2.set_xlabel("Minute")
ax2.grid(True, alpha=0.3)
ax2.legend(bbox_to_anchor=(1, 1), loc='upper left')

plt.tight_layout()
plt.show()